**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_numerical -p 8888:8888 -d palmoreck/jupyterlab_numerical:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_numerical
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

---

Nota generada a partir de [liga1](https://drive.google.com/file/d/1zCIHNAxe5Shc36Qo0XjehHgwrafKSJ_t/view), [liga2](https://drive.google.com/file/d/1RMwUXEN_SOHKue-J9Cx3Ldvj9bejLjiM/view).

In [1]:
!pip3 install --user -q cvxpy

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import os

In [2]:
cur_directory = os.getcwd()

In [3]:
dir_alg_python = '/algoritmos/Python'

In [4]:
os.chdir(cur_directory + dir_alg_python)

In [5]:
import math

import numpy as np

from utils import compute_error

from algorithms_for_cieco import path_following_method_infeasible_init_point


# Está lista la implementación para puntos iniciales no factibles en $Ax=b$ pero no para la parte de las desigualdades $f_i(x) <0 \quad \forall i=1,\dots,m$

# Primer ejemplo

$$ \min \quad x_1^2 + x_2^2 + x_3^2 + x_4^2 -2x_1-3x_4$$

$$\text{sujeto a: } $$

$$
\begin{array}{c}
2x_1 + x_2 + x_3 + 4x_4 = 7 \\
x_1 + x_2 + 2x_3 + x_4 = 6
\end{array}
$$

$$x_1, x_2, x_3, x_4 \geq 0$$

## Infeasible for Ax=b

In [6]:
fo = lambda x: x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2-2*x[0]-3*x[3]

In [7]:
const = {0: lambda x: -x[0],
         1: lambda x: -x[1],
         2: lambda x: -x[2],
         3: lambda x: -x[3]
        }

In [8]:
A= np.array([[2,1,1,4],
             [1,1,2,1]])

In [9]:
b=np.array([7,6])

In [10]:
x_ast=np.array([1.1232876712328763,0.6506849315068493,
                1.8287671232876714,0.5684931506849317])

In [11]:
x_0 = np.array([1,1,1,1],dtype=float)

In [12]:
x_0

array([1., 1., 1., 1.])

In [13]:
nu_0 = np.array([0,0], dtype=float)

In [14]:
p_ast=fo(x_ast)

In [15]:
p_ast

1.4006849315068495

In [21]:
tol_outer_iter = 1e-6
tol=1e-8
tol_backtracking=1e-8
maxiter=30
mu=10

In [22]:
[x,iter_barrier,t] = path_following_method_infeasible_init_point(fo, A, b,
                                                                 const,
                                                               x_0, nu_0, tol,
                                                               tol_backtracking, x_ast, p_ast, maxiter,
                                                               mu, tol_outer_iter = tol_outer_iter 
                                                               )

Outer iterations of path following method
Mu value: 1.00e+01
Outer iteration	LogBarrier 	t_log_barrier	Stopping criteria
1		-1.67e+00	1.67e+00	2.40e+00
----------------------------------------------------------------------------------------
I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.41e+00	4.36e+00	-2.49e+00	4.34e-01	1.71e+00	---		1.00e+00
1	8.88e-16	3.50e+00	1.02e-01	9.76e-02	3.64e-02	1.00e+00	1.00e+00
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
Beginning Newtons method for feasible initial point
I	Norm gfLogBarrier 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0		7.49e+00		1.40e-02	9.76e-02	3.64e-02	---		1.86e+00
1		7.49e+00		5.83e-08	8.44e-02	2.72e-02	1.00e+00	1.86e+00
2		7.49e+00		5.83e-08	8.44e-02	2.72e-02	9.09e-13	1.86e+00
Error of x with respect to x_ast: 8.44e-02
Approximate 

In [23]:
[x,iter_barrier,t]

[array([1.12328763, 0.6506851 , 1.82876707, 0.56849315]),
 261,
 16661911.55492154]

In [24]:
compute_error(x_ast,x)

7.734328493620186e-08

In [25]:
x_ast

array([1.12328767, 0.65068493, 1.82876712, 0.56849315])

# Comparación con [cvxpy](https://github.com/cvxgrp/cvxpy)

In [26]:
import cvxpy as cp

In [27]:
x1 = cp.Variable()
x2 = cp.Variable()
x3 = cp.Variable()
x4 = cp.Variable()


In [28]:
# Create two constraints.
constraints = [2*x1+x2+x3+4*x4-7 == 0,x1+x2+2*x3+x4-6 == 0,x1>=0,x2>=0,x3>=0,x4>=0]

# Form objective.

obj = cp.Minimize(x1**2+x2**2+x3**2+x4**2-2*x1-3*x4)

In [29]:
# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.

1.4006849315068515

In [30]:
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x1.value, x2.value, x3.value,x4.value)

status: optimal
optimal value 1.4006849315068515
optimal var 1.1232876712328763 0.6506849315068494 1.8287671232876717 0.5684931506849316


# Segundo ejemplo

$$\min 2x_1 + 5x_2$$

$$\text{sujeto a: }$$

$$
\begin{array}{c}
6-x_1-x_2 \leq 0 \\
-18 + x_1 +2x_2 \leq 0\\
x_1, x_2 \geq 0
\end{array}
$$

In [31]:
fo = lambda x: 2*x[0] + 5*x[1]

In [32]:
const = {0: lambda x: 6-x[0]-x[1],
         1: lambda x: -18+x[0]+2*x[1],
         2: lambda x: -x[0],
         3: lambda x: -x[1]
        }

In [33]:
A=np.array([0,0],dtype=float)
b = 0

In [34]:
x_ast = np.array([6,0], dtype=float)

In [35]:
x_0 = np.array([4,4], dtype=float)

In [36]:
p_ast=fo(x_ast)

In [37]:
p_ast

12.0

In [38]:
nu_0 = np.array([0,0], dtype=float)

In [39]:
tol_outer_iter = 1e-3
tol=1e-8
tol_backtracking=1e-8
maxiter=30
mu=10

In [40]:
[x,iter_barrier,t] = path_following_method_infeasible_init_point(fo, A, b,
                                                                 const,
                                                               x_0, nu_0, tol,
                                                               tol_backtracking, x_ast, p_ast, maxiter,
                                                               mu, tol_outer_iter = tol_outer_iter 
                                                               )

Outer iterations of path following method
Mu value: 1.00e+01
Outer iteration	LogBarrier 	t_log_barrier	Stopping criteria
1		1.23e+01	2.50e-01	1.60e+01
----------------------------------------------------------------------------------------
I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.00e+00	8.37e-01	5.59e+00	7.45e-01	1.33e+00	---		9.46e+00
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
Beginning Newtons method for feasible initial point
I	Norm gfLogBarrier 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0		8.37e-01		5.59e+00	7.45e-01	1.33e+00	---		9.46e+00
1		8.37e-01		2.61e-02	2.15e-01	5.37e-01	5.00e-01	9.46e+00
2		8.37e-01		2.61e-02	2.15e-01	5.37e-01	1.78e-15	9.46e+00
Error of x with respect to x_ast: 2.15e-01
Approximate solution: [6.88318309 0.93672033]
Backtracking value less than tol_

/datos/MNO_desde_2018/ramas_repo/mno-master/temas/IV.optimizacion_convexa_y_machine_learning/algoritmos/Python/utils.py:55: RuntimeWarning: invalid value encountered in log
  eval_f_const_inequality = np.log(-eval_f_const_inequality)


In [41]:
[x,iter_barrier,t]

[array([6.00008070e+00, 7.00989697e-05]), 105, 25000.0]

In [42]:
compute_error(x_ast,x)

1.7815100934691582e-05

# Comparación con [cvxpy](https://github.com/cvxgrp/cvxpy)

In [43]:
x1 = cp.Variable()
x2 = cp.Variable()


In [44]:
# Create two constraints.
constraints = [6-x1-x2 <= 0,-18+x1+2*x2<=0,x1>=0,x2>=0]

# Form objective.

obj = cp.Minimize(2*x1+5*x2)



In [45]:
# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.


12.0000000016275

In [46]:
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x1.value, x2.value)

status: optimal
optimal value 12.0000000016275
optimal var 6.000000000175689 2.552244387851183e-10


**Referencias:**

* S. P. Boyd, L. Vandenberghe, Convex Optimization, Cambridge University Press, 2009.